In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
bp =  '/content/drive/MyDrive/NLP_project'

In [3]:
import glob  # Finds all the fileLocationnames matching a specified pattern, 
             # typically specified with regex (re) 
import re
import pandas as pand
from nltk import pos_tag
from nltk.chunk.regexp import RegexpChunkParser, ChunkRule, RegexpParser
from nltk.tree import Tree
import numpy as np

pand.set_option('max_colwidth', None)  # Remove any limitation on length 
                                     # of text displayed in a cell
#pand.set_option('max_rows', 300)  # Display up to 300 rows in a dataset

In [19]:
data_dir = '/content/drive/MyDrive/NLP_project/data/'  # Path to directory containing .con and .txt files
output_dir = data_dir+'output/'

listOfCon = glob.glob(dataFold+'concepts/*.txt.con')  # Make list of concept files
listOfTxt = glob.glob(dataFold+'docs/*.txt')  # Make list of docus
stringBase = 'clinical-'

In [5]:
conFileIds = []
txtFileIds = []

# Use regex to create doc id 

for con in listOfCon:
    filID = re.findall(r'\d+', con)[0]
    conFileIds.append(filID)
for doc in listOfTxt:
    filID = re.findall(r'\d+', doc)[0]
    txtFileIds.append(filID)
    
conFileIds = tuple(sorted(conFileIds)) 
txtFileIds = tuple(sorted(txtFileIds))

intersection = list(set(conFileIds) & set(txtFileIds))
if len(intersection) == len(conFileIds):
    print("Count of concept files with corresponding doc:", len(intersection))

Count of concept files with corresponding doc: 0


In [ ]:
# typeAs=[]

# for doc in listOfCon:
#     for row in doc:
#         print(row)
#         typeAs.append(row.split('"')[-2])

# typeAs = set(typeAs)

# count_dict = {el: 0 for el in typeAs}

# for doc in listOfCon:
#     for singleSent in doc:
#         for key in count_dict.keys():
#             if 't="'+key+'"' in singleSent:
#                 count_dict[key] += 1
# count_dict

In [6]:
# Build annotation and entry corpora

listOfCon = []
listOfTxt = []

for filID in conFileIds:
    fileLocation = dataFold + "concepts/" + stringBase + filID +".txt.con"
    with open(fileLocation) as f:
        dataInFile = f.read().splitsingleSents()
        listOfCon.append(dataInFile)

    fileLocation = dataFold + "docs/" + stringBase + filID +".txt"
    with open(fileLocation) as f:
        dataInFile = f.read().splitsingleSents()
        listOfTxt.append(dataInFile)

In [7]:
columnsOfEntries = ["id", "row", "offset", "word"]
dataframesOfEntries = pand.DataFrame(columns=columnsOfEntries)

columnsForAnnotations = ["id", "NER_tag", "row", "offset", "length"]
dataframesOfAnnotations = pand.DataFrame(columns=columnsForAnnotations)

In [8]:
dataframesOfAnnotations = pand.DataFrame(columns=columnsForAnnotations)  # Reset df
listStoreTemp = []  # Set up variable to hold row info

for i, docu in enumerate(listOfCon):
    
    for row in docu:
        row = row.split("||")
        dataText = row[0]
        dataType = row[1]
        
        text = dataText.split('"')[1]
        
        beginOffs = dataText.split(' ')[-2]
        endingOff = dataText.split(' ')[-1]
        
        singleSent = beginOffs.split(':')[0] # Given one sentence to singleSent, 
                                          # singleSent number will be the same for beginOffs and endingOff
        
        beginOffsWord = int(beginOffs.split(':')[1])
        endingOffsWord = int(endingOff.split(':')[1])
        length = endingOffsWord-beginOffsWord +1
        
        typeA = dataType.split('"')[-2]
        
        # Split text into tokens with IOB tags
        initFlag = True  # Set up flag to id start of text
        tagBio = "B-"
        if length > 1:  # Isolate text with multiple tokens 
            for offset in range(beginOffsWord, endingOffsWord+1):
                if initFlag:
                    labelOfTag = tagBio + typeA # Set tag for initFlag word to start with B-
                    initFlag = False  # Change flag
                else:
                    labelOfTag = labelOfTag.replace("B-", "I-")
                listStoreTemp.append([conFileIds[i], labelOfTag, singleSent, offset, 1])                
        else:
            listStoreTemp.append([conFileIds[i], tagBio + typeA, singleSent, beginOffsWord, length])
        
dataframesOfAnnotations = pand.DataFrame(listStoreTemp, columns=columnsForAnnotations)
dataframesOfAnnotations = dataframesOfAnnotations.drop(columns=["length"])

dataframesOfAnnotations.head(50)

,id,NER_tag,row,offset


In [9]:
dataframesOfEntries = pand.DataFrame(columns=columnsOfEntries)  # Reset df
listStoreTemp = []

for iDoc, docu in enumerate(listOfTxt):
    
    listStoreTemp.append([0, 0, 0, "-DOCSTART-"])
    listStoreTemp.append([0, 0, 0, "-EMPTYsingleSent-"])
    
    for ithRow, row in enumerate(docu):
        splitRow = row.split(" ")
        for ithWord, word in enumerate(splitRow):
            word = word.replace("\t", "")
            ithWordd = conFileIds[iDoc]
            rowWord = ithRow+1  # 1-based indexing 
            offWord = ithWord # 0-based indexing
            
            if len(word) > 0 and "|" not in word:
                listStoreTemp.append([ithWordd, rowWord, offWord, word])
        
    listStoreTemp.append([0, 0, 0, "-EMPTYsingleSent-"])

dataframesOfEntries = pand.DataFrame(listStoreTemp, columns=columnsOfEntries)

countOfNER = [1 for i in dataframesOfAnnotations["NER_tag"] if "B-" in i]
print(len(countOfNER), "named entities")

0 named entities


In [10]:
dataframesOfAnnotations[['id', 'row', 'offset']] = dataframesOfAnnotations[['id', 'row', 'offset']].apply(pand.to_numeric)
dataframesOfAnnotations['NER_tag'] = dataframesOfAnnotations["NER_tag"].astype(str)
dataframesOfEntries[['id', 'row', 'offset']] = dataframesOfEntries[['id', 'row', 'offset']].apply(pand.to_numeric)
dataframesOfEntries["word"] = dataframesOfEntries["word"].astype(str)

dataframeRES = pand.merge(dataframesOfEntries, dataframesOfAnnotations, how="left", on=['id', 'row', 'offset'])

# Check for NaNs (should be only in NER_tag, where NaNs will be replaced with "O" (outside))
print("Columns with missing data:\n", dataframeRES.isna().any())

Columns with missing data:
 word       False
id         False
NER_tag    False
row        False
offset     False
dtype: bool


In [11]:
dataframeRES = dataframeRES.fillna("O")
dataframeRES = dataframeRES.drop(columns=["id", "row", "offset"])

countOfNER = [1 for i in dataframeRES["NER_tag"] if "B-" in i]
print(len(countOfNER), "named entities and", dataframeRES.shape[0], "tokens")

0 named entities and 0 tokens


In [12]:
import nltk
nltk.download('averaged_perceptron_tagger')
text = dataframeRES["word"].tolist()
positionOfText = pos_tag(text)
positionOfText_list = [i[1] for i in positionOfText]
print(len(positionOfText_list), 'tokens')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


0 tokens


In [13]:
# Chunk noun phrases

ruleInit = ChunkRule("<DT>?<JJ.*>*<NN.*>+", "More complete chunk NP sequences")
parserChunkNoun = RegexpChunkParser([ruleInit],chunk_label='NP')
resultTreeNoun = parserChunkNoun.parse(positionOfText)

chunkTagNoun = []

for i in resultTreeNoun:
    if isinstance(i, Tree):
        for j in range(0, len(i)):
            if j == 0:
                # print("B-" + i.label())
                chunkTagNoun.append("B-" + i.label())
            else:
                chunkTagNoun.append("I-" + i.label())
                # print("I-" + i.label())
    else:
        # print("O")
        chunkTagNoun.append("O")

len(chunkTagNoun) == dataframeRES.shape[0]  # check that chunk col has same length

True

In [14]:
# Chunk verb phrases

secRule = ChunkRule("<VBD|IN|\.>", "Verb phrases")
parserChunkVerb = RegexpChunkParser([secRule],chunk_label='VP')
resultTreeVerb = parserChunkVerb.parse(positionOfText)

chunkTagVerb = []

for i in resultTreeVerb:
    if isinstance(i, Tree):
        for j in range(0, len(i)):
            if j == 0:
                # print("B-" + i.label())
                chunkTagVerb.append("B-" + i.label())
            else:
                chunkTagVerb.append("I-" + i.label())
                # print("I-" + i.label())
    else:
        # print("O")
        chunkTagVerb.append("O")
        
len(chunkTagNoun) == dataframeRES.shape[0] == len(chunkTagVerb)

True

In [15]:
# Augment chunk tags with verb phrase tags

for i, entry in enumerate(chunkTagNoun):
    if entry == "O":
        chunkTagNoun[i] = chunkTagVerb[i]
        
dataframeRES["POS_tag"] = positionOfText_list
dataframeRES["chunk_tag"] = chunkTagNoun

dataframeRES = dataframeRES[['word', 'POS_tag', 'chunk_tag', 'NER_tag']]  # Reorder columns
dataframeRES[['word', 'POS_tag', 'chunk_tag', 'NER_tag']] = dataframeRES[['word', 'POS_tag', 'chunk_tag', 'NER_tag']].astype(str)


In [16]:
opDataframe = dataframeRES.copy()

# Identify idx at end of sentences
idx = opDataframe[opDataframe['word'] == '.'].index.values.tolist()
print(len(idx), 'total sentences')

0 total sentences


In [17]:
# Create a df of blank rows to insert after end of sentences
newDataFrame = pand.DataFrame(index=[el+0.5 for el in idx]) 
opDataframe = pand.concat([opDataframe, newDataFrame])
opDataframe = opDataframe.sort_index()
opDataframe = opDataframe.reset_index(drop=True)
opDataframe.fillna("", inplace=True)
opDataframe[opDataframe['word'] == ""].head()

,word,POS_tag,chunk_tag,NER_tag


In [23]:
train = int(round(opDataframe.shape[0]*.7,0))  
dev = int(round(opDataframe.shape[0]*.85,0))
trainResDataframe = opDataframe.iloc[:train]
validResDataframe = opDataframe.iloc[train:dev]
testResDataframe = opDataframe.iloc[dev:]

np.savetxt(output_dir+"train.txt", trainResDataframe.values, fmt="%s")
np.savetxt(output_dir+"valid.txt", validResDataframe.values, fmt="%s")
np.savetxt(output_dir+"test.txt", testResDataframe.values, fmt="%s")

FileNotFoundError: ignored

In [ ]:
trainResDataframe.to_csv(output_dir+"train.csv")
validResDataframe.to_csv(output_dir+"valid.csv")
testResDataframe.to_csv(output_dir+"test.csv")